In [1]:
# Copies the model weights from google drive into colab
%cp -r "/content/drive/MyDrive/hf_llama_2_7b" "/content/"

In [2]:
%cp -r "/content/drive/MyDrive/outputs" "/content/"

In [18]:
# Installs required python packages
!pip install torch accelerate bitsandbytes datasets transformers peft trl scipy astrapy pandas openai ftfy

In [36]:
# Import required python packages
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, set_seed, Trainer, TrainingArguments, BitsAndBytesConfig, DataCollatorForLanguageModeling
from peft import PeftModel, PeftConfig
import bitsandbytes as bnb
from torch import cuda, bfloat16
import transformers
import torch
import torch.nn as nn
from google.colab import userdata
from astrapy.db import AstraDBCollection, AstraDB
from datasets import Dataset
from pprint import pprint
import pandas as pd
import os
from dotenv import load_dotenv
import ftfy
import pprint

In [5]:
# Uses GPU for processing if a CUDA device is available
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [6]:
# Pulls all our instruction data from Astra
token = userdata.get('astra_token')
endpoint = userdata.get('astra_endpoint')

collection_name = "test_instructions"
rag_collection_name = "article_embeddings"

# API key for OpenAI
OPENAI_API_KEY = os.getenv("openai_key")

# Client for OpenAI API
client = OpenAI(api_key = OPENAI_API_KEY)

# Initialize AstraDB instance and AstraDBCollection instances for input and output collections
astra_db = AstraDB(token=token, api_endpoint=api_endpoint)
collection = AstraDBCollection(collection_name=in_collection_name, astra_db=astra_db)

# Create the output collection
astra_db.create_collection(collection_name=out_collection_name, dimension=1536)
rag_collection = AstraDBCollection(collection_name=out_collection_name, astra_db=astra_db)

nextPageState = ""
raw_dataset = []
expected_columns = ['_id','instruction', 'input', 'output']

def check_expected_columns(raw_instruction):
  if all(column in raw_instruction for column in expected_columns):
    return True
  else:
    return False

while nextPageState != None:
  if nextPageState == "":
    data = collection.find()
    nextPageState = data['data']['nextPageState']
    raw_instructions = [instruction for instruction in data['data']['documents'] if check_expected_columns(instruction)]
    raw_dataset.extend(raw_instructions)
  else:
    data = collection.find(options={"pageState":nextPageState}, sort = None)
    nextPageState = data['data']['nextPageState']
    raw_instructions = [instruction for instruction in data['data']['documents'] if check_expected_columns(instruction)]
    raw_dataset.extend(raw_instructions)

print(raw_dataset[0])
print(len(raw_dataset))

{'_id': '0858ca63-d084-4a0b-98ca-63d0844a0bdf', 'instruction': 'How do I set up a Cassandra database using DataStax Astra for a REST API application?', 'input': "To implement a Cassandra database with DataStax Astra for a REST API application, follow these steps: 1. Register or sign in to Astra using your Github, Google accounts, or email. Fill the Create New Database Form with the recommended values for Compute Size, region, database name, keyspace name, user name, and password. Launch the database and wait for it to initialize. 2. Copy your credentials by managing your organization, adding a service account, and copying the credentials. 3. Run the application in Gitpod by pasting the service account credentials. 4. Test the application by using the TodoMVC community contribution. Verify the application's functionality by running tests on the TodoBackEnd Spec Runner and Web Client.", 'output': 'To set up a Cassandra database using DataStax Astra, first create a DB-as-a-service by regi

In [ ]:
def get_similar_snippets(context):
    embedding = client.embeddings.create( input=context, model="text-embedding-ada-002").data[0].embedding
    similar_rows = rag_collection.vector_find(embedding, limit=3)
    extra_context = "\n".join([ row['content'] for row in similar_rows['data']['documents']])
    formatted_extra_context = f"### Extra Context: \n{extra_context}\n"

In [35]:
# Turns separated instruction dicts from Astra into a dataset of combined instructions
def build_instruction_prompt(record):
    start = "Read the Instruction below and provide an answer."
    question = f"### INSTRUCTION:\n{record['instruction']}\n\n"
    response = f"### Context:\n{record['input']}\n"
    
    answer = f"### Response: "

    instruction_and_context = "\n".join([part for part in [start, question, response] if part])
    extra_context = get_similar_snippets(instruction_and_context)
    parts = [part for part in [start, question, response, extra_context, answer] if part]

    formatted_prompt = "\n\n".join(parts)
    formatted_prompt = formatted_prompt.replace('\\n', '\n')

    record["text"] = formatted_prompt
    return record

p = build_instruction_prompt(raw_dataset[0])
#pprint(p["text"])
combined_dataset = list(map(build_instruction_prompt, raw_dataset))
#pprint(combined_dataset[0:2])

dataframe = pd.DataFrame(data=combined_dataset, dtype='string')
dataframe.info()
dataset = Dataset.from_pandas(dataframe)
pprint(dataset[2])


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1847 entries, 0 to 1846
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          1847 non-null   string
 1   instruction  1847 non-null   string
 2   input        1847 non-null   string
 3   output       1847 non-null   string
 4   article_id   1847 non-null   string
 5   text         1847 non-null   string
dtypes: string(6)
memory usage: 86.7 KB
{'_id': '06711797-ad1c-48f1-b117-97ad1cb8f1a3',
 'article_id': '89266c85-be29-449f-b017-6c8b46913a71',
 'input': 'The DataStax Apache Kafka Connector synchronizes records from a '
          'Kafka topic with table rows in supported databases like DataStax '
          'Astra, DataStax Enterprise, and Apache Cassandra. It operates as '
          'open-source software within the Kafka Connect framework. Users need '
          'to be cautious about ensuring proper ordering of records by '
          'utilizing Kafka record ti

In [39]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [59]:
# Set quantization settings
model_id = "hf_llama_2_7b"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)
model_config = AutoConfig.from_pretrained(model_id)

In [62]:
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [66]:
test = dataset[0]
batch = tokenizer(test['text'], return_tensors='pt').to('cuda')
with torch.cuda.amp.autocast():
  original_tokens = model.generate(**batch, max_new_tokens=200)

original_output = tokenizer.decode(original_tokens[0], skip_special_tokens=True)
original_response = original_output[len(test['text']):].split("\n\n\n\n")[0]
print("Original: \n")
print(original_response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Original: 

Read the Instruction below and provide an answer.

### INSTRUCTION:
How do I set up a Cassandra database using DataStax Astra for a REST API application?



### Context:
To implement a Cassandra database with DataStax Astra for a REST API application, follow these steps: 1. Register or sign in to Astra using your Github, Google accounts, or email. Fill the Create New Database Form with the recommended values for Compute Size, region, database name, keyspace name, user name, and password. Launch the database and wait for it to initialize. 2. Copy your credentials by managing your organization, adding a service account, and copying the credentials. 3. Run the application in Gitpod by pasting the service account credentials. 4. Test the application by using the TodoMVC community contribution. Verify the application's functionality by running tests on the TodoBackEnd Spec Runner and Web Client.


### Response: 


### Hint:


### Solution:


### Explanation:


### References:




In [72]:
collection.update_one(
    filter={"_id": test['_id']},
    update={"$set": {"rag_llm_response": original_response}},
)

document = collection.find_one(filter={"_id": test['_id']})

In [83]:
idx_min = 0
idx_max = 500
partial_dataset = dataset.filter(lambda example, idx: idx >= idx_min and idx < idx_max, with_indices=True)
for row in partial_dataset:
  print(row)
  batch = tokenizer(row['text'], return_tensors='pt').to('cuda')
  with torch.cuda.amp.autocast():
    original_tokens = model.generate(**batch, max_new_tokens=200)

    original_output = tokenizer.decode(original_tokens[0], skip_special_tokens=True)
    original_response = original_output[len(row['text']):].split("\n\n\n\n")[0]

    collection.update_one(
      filter={"_id": row['_id']},
      update={"$set": {"original_response": original_llm_response}},
    )

Filter:   0%|          | 0/1847 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'_id': '0858ca63-d084-4a0b-98ca-63d0844a0bdf', 'instruction': 'How do I set up a Cassandra database using DataStax Astra for a REST API application?', 'input': "To implement a Cassandra database with DataStax Astra for a REST API application, follow these steps: 1. Register or sign in to Astra using your Github, Google accounts, or email. Fill the Create New Database Form with the recommended values for Compute Size, region, database name, keyspace name, user name, and password. Launch the database and wait for it to initialize. 2. Copy your credentials by managing your organization, adding a service account, and copying the credentials. 3. Run the application in Gitpod by pasting the service account credentials. 4. Test the application by using the TodoMVC community contribution. Verify the application's functionality by running tests on the TodoBackEnd Spec Runner and Web Client.", 'output': 'To set up a Cassandra database using DataStax Astra, first create a DB-as-a-service by regi

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'_id': '598f1589-8805-4c42-8f15-8988059c425e', 'instruction': 'Explain the key factors to consider when denormalizing data in Cassandra for better performance.', 'input': "In Cassandra, denormalization is crucial for faster access, where data duplication is traded for reduced read latency. When working with Cassandra's data model, consider denormalizing immutable data for optimal performance. For mutable data that necessitates denormalization, two primary strategies are typically employed: normalization with extra reads or denormalization with read-before-write and manual update handling. To mitigate the challenges of denormalization, Cassandra offers materialized views designed to alleviate developers' burdens, though not eliminating all denormalization overhead. When creating materialized views in Cassandra, ensure that the view includes all columns of the base table's primary key and that the primary key columns contain no null values. These views help in presenting data from the b

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'_id': '43637db4-a360-4f2f-a37d-b4a360ff2f52', 'instruction': "Summarize the main idea of the 'Data partitioning and denormalization' section.", 'input': "To assess Cassandra performance, it's logical to start at the beginning of data's path and first look at its efficiency while distributing and duplicating data. Cassandra uses consistent hashing for data partitioning and practices data replication and partitioning. Denormalization and data duplication are encouraged in Cassandra for optimizing read requests. While this strategy enhances read performance, it can impact write performance by requiring multiple writes. However, consistent hashing ensures efficient data partitioning, linear scalability, and fault tolerance in the cluster. Data consistency and indexing can be challenging in Cassandra, with secondary indexes and materialized views affecting write and read operations.", 'output': "The 'Data partitioning and denormalization' section emphasizes how Cassandra efficiently distr

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'_id': 'd02a53ae-b1e8-4d63-aa53-aeb1e84d63f8', 'instruction': 'How does Cassandra ensure fault tolerance and availability in its architecture?', 'input': 'Cassandra is known for its fault tolerance and availability features. It is designed to be massively scalable, high performance, always-on, and masterless. It offers automatic replication, supports multi-datacenter setups, and has a decentralized architecture eliminating single points of failure. In case of regional outages, Cassandra clusters can survive, and new nodes can automatically join the cluster. DataStax drivers can discover and utilize new nodes seamlessly.', 'output': 'Cassandra ensures fault tolerance and availability through its decentralized, masterless architecture that supports automatic replication, multi-datacenter setups, and the ability for new nodes to join the cluster seamlessly. This design helps Cassandra maintain high availability and survive regional outages by eliminating single points of failure.', 'arti

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
